# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [3]:
import numpy as np
import pandas as pd
from collections import deque

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
class Node(object):
    def __init__(self, X=None, depth=0):
        self.X = X
        self.depth = depth
        self.split_param = None
        self.target_value = None
        self.left_child = None
        self.right_child = None

    def target(self):
        if self.target_value is None:
            targets, counts = np.unique(self.X.iloc[:, -1], return_counts=True)
            best_target = (-1, float('-inf'))
            for target, prop in zip(targets, counts):
                if prop > best_target[1]:
                    best_target = (target, prop)
            self.target_value = best_target[0]
        return self.target_value


def entropy(y):
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy_val = sum(probabilities * -np.log2(probabilities))

    return entropy_val


class DecisionTreeClassifier(object):

    def __init__(self, max_depth=-1, min_samples_leaf=1, max_leaf_nodes=-1, purity=False):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_leaf_nodes = max_leaf_nodes
        self.purity = purity

        self.root = Node()
        self.size = 1
        self.next_nodes = deque()

    def split(self, X, j, t):
        sample1 = X[X.iloc[:, j] < t]
        sample2 = X[X.iloc[:, j] >= t]

        return sample1, sample2

    def calc_ig(self, r_v, r_left, r_right):
        r_left_prop = len(r_left) / len(r_v)
        r_right_prop = len(r_right) / len(r_v)
        return entropy(r_v) - \
               (r_left_prop * entropy(r_left) + r_right_prop * entropy(r_right))

    def find_best_split(self, X):
        best_values = (float('-inf'), -1, -1)
        param_n = X.shape[1] - 1
        for j in range(param_n):
            min_value = X.iloc[:, j].min()
            max_value = X.iloc[:, j].max()
            values = np.linspace(min_value, max_value, num=10)
            for t in values:
                r_left, r_right = self.split(X, j, t)
                ig = self.calc_ig(X.iloc[:, -1], r_left.iloc[:, -1], r_right.iloc[:, -1])
                if ig > best_values[0]:
                    best_values = (ig, j, t)
        return best_values[1], best_values[2]

    def check_purity(self, X):
        targets = np.unique(X)
        return len(targets) == 1

    def fit(self, X_train, y_train):
        X = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1))))
        self.root.X = X
        self.next_nodes.append(self.root)

        while len(self.next_nodes) > 0:
            cur_node = self.next_nodes.popleft()
            if cur_node.depth == self.max_depth:
                continue
            if cur_node.X.shape[0] <= self.min_samples_leaf:
                continue
            if self.purity and self.check_purity(cur_node.X.iloc[:, -1]):
                continue

            j, t = self.find_best_split(cur_node.X)
            cur_node.split_param = (j, t)
            r_left, r_right = self.split(cur_node.X, j, t)

            left_node = Node(r_left, cur_node.depth + 1)
            cur_node.left_child = left_node
            self.next_nodes.append(left_node)
            self.size += 1
            if self.size == self.max_leaf_nodes:
                self.next_nodes.clear()
                break

            right_node = Node(r_right, cur_node.depth + 1)
            cur_node.right_child = right_node
            self.next_nodes.append(right_node)
            self.size += 1
            if self.size == self.max_leaf_nodes:
                self.next_nodes.clear()
                break

    def calc_final_node(self, node):
        if node.left_child is None and node.right_child is None:
            node.target()

        if node.left_child is not None:
            self.calc_final_node(node.left_child)

        if node.right_child is not None:
            self.calc_final_node(node.right_child)

    def get_target(self, obj, node):
        if node.right_child is None and node.left_child is None:
            return node.target()

        j, t = node.split_param

        if obj[j] >= t:
            return self.get_target(obj, node.right_child)
        else:
            return self.get_target(obj, node.left_child)

    def predict(self, X_test):
        return [self.get_target(X_test[i, :], self.root) for i in range(len(X_test))]    

In [ ]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)

tree = DecisionTreeClassifier(purity=True)
tree.fit(X_train, y_train)
print('Accuracy with purity -', accuracy_score(tree.predict(X_test), y_test))

tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)
print('Accuracy with max depth 5 -', accuracy_score(tree.predict(X_test), y_test))

tree = DecisionTreeClassifier(max_leaf_nodes=15)
tree.fit(X_train, y_train)
print('Accuracy with 15 leaf nodes -', accuracy_score(tree.predict(X_test), y_test))

Accuracy with purity - 0.8333333333333334
Accuracy with max depth 5 - 0.8666666666666667
Accuracy with 15 leaf nodes - 0.9


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator
import numpy as np
from random import randrange
from sklearn.preprocessing import OneHotEncoder


class RandomForest(BaseEstimator):
    estimators = list()

    def __init__(self, base_estimator=DecisionTreeClassifier, max_features='sqrt',
                 n_estimators=10, max_depth=None, min_samples_leaf=1, max_leaf_nodes=None, criterion='gini'):
        self.base_estimator = base_estimator
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_leaf_nodes = max_leaf_nodes
        self.criterion = criterion

    def get_boostrap_sample(self, sample):
        new_sample = np.empty([sample.shape[0], sample.shape[1]])
        n_sample = sample.shape[0]
        for i in range(n_sample):
            index = randrange(n_sample)
            new_sample[i] = sample[index]
        return new_sample

    def fit(self, X, y):
        for i in range(self.n_estimators):
            Xy = np.hstack((X, y.reshape(-1, 1)))
            new_Xy = self.get_boostrap_sample(Xy)
            new_X_train, new_y_train = new_Xy[:, :-1], new_Xy[:, -1].flatten()
            estimator = self.base_estimator(max_features=self.max_features, max_depth=self.max_depth,
                                            min_samples_leaf=self.min_samples_leaf, max_leaf_nodes=self.max_leaf_nodes,
                                            criterion=self.criterion)
            estimator.fit(new_X_train, new_y_train)
            self.estimators.append(estimator)
        return self

    def predict_obj(self, pred_obj):
        values, counts = np.unique(pred_obj, return_counts=True)
        ind = np.argmax(counts)
        return values[ind]

    def predict(self, X_test):
        predictions = np.array([estimator.predict(X_test) for estimator in self.estimators])
        return [self.predict_obj(predictions[:, i]) for i in range(X_test.shape[0])]

    def score(self, X_test, y_test):
        pred_y = self.predict(X_test)
        return accuracy_score(pred_y, y_test)

    def get_params(self, deep=False):
        return {'base_estimator': self.base_estimator,
                'max_features': self.max_features,
                'n_estimators': self.n_estimators,
                'max_depth': self.max_depth,
                'min_samples_leaf': self.min_samples_leaf,
                'max_leaf_nodes': self.max_leaf_nodes,
                'criterion': self.criterion}

    def set_params(self, params):
        for parameter, value in params:
            setattr(self, parameter, value)
        return self

    def get_feature_informativeness(self, X, y):
        informativeness_matrix = np.empty((self.n_estimators, X.shape[1]))
        for i in range(self.n_estimators):
            Xy = np.hstack((X, y.reshape(-1, 1)))
            new_Xy = self.get_boostrap_sample(Xy)
            new_X_train, new_y_train = new_Xy[:, :-1], new_Xy[:, -1].flatten()
            estimator = self.base_estimator(max_features=self.max_features, max_depth=self.max_depth,
                                            min_samples_leaf=self.min_samples_leaf, max_leaf_nodes=self.max_leaf_nodes,
                                            criterion=self.criterion)
            estimator.fit(new_X_train, new_y_train)
            oob_Xy = np.array([obj for obj in Xy if not any(np.equal(new_Xy, obj).all(1))])
            oob_X_test, oob_y_test = oob_Xy[:, :-1], oob_Xy[:, -1].flatten()
            pred_y = estimator.predict(oob_X_test)
            initial_q = accuracy_score(pred_y, oob_y_test)
            for j in range(oob_X_test.shape[1]):
                noise = np.array([randrange(-1000, 1000) for _ in range(oob_X_test.shape[0])])
                oob_X_test_copy = oob_X_test.copy()
                oob_X_test_copy[:, j] = noise
                noise_pred_y = estimator.predict(oob_X_test_copy)
                noise_q = accuracy_score(noise_pred_y, oob_y_test)
                informativeness_matrix[i, j] = initial_q - noise_q
        return np.mean(informativeness_matrix, axis=0).argsort()[::-1]


In [5]:
import itertools as it
from sklearn.model_selection import cross_val_score


class GridSearch(object):
    best_score_ = float("-inf")
    best_params_ = None

    def __init__(self, checked_estimator, params, scoring, cv):
        self.checked_estimator = checked_estimator
        self.params = params
        self.scoring = scoring
        self.cv = cv

    def fit(self, X, y):
        param_combs = [[list(el) for el in zip(self.params.keys(), comb)]
                       for comb in it.product(*(self.params[key] for key in self.params.keys()))]
        for params in param_combs:
            estimator = self.checked_estimator()
            estimator.set_params(params)
            score = cross_val_score(estimator, X, y, scoring=self.scoring, cv=self.cv).mean()
            if self.best_score_ < score:
                self.best_score_ = score
                self.best_params_ = params
        return self

In [6]:
df = pd.read_csv('churn.csv')

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


При обучении модели не будем учитывать признаки RowNumber, CustomerId Surname, так как они не влияют на то, покинет ли клиент банк (являются по сути шумовыми).

In [8]:
categorical_data = df[['Geography', 'Gender']]
numertical_binary_data = df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary','HasCrCard', 'IsActiveMember']].to_numpy()
y = df['Exited'].to_numpy()
encoder = OneHotEncoder(handle_unknown='ignore')
categorical_data = encoder.fit_transform(categorical_data).toarray()
X = np.hstack((numertical_binary_data, categorical_data))

In [9]:
print(cross_val_score(RandomForest(), X, y, cv=3).mean())

0.9109063275853679


Найдем науличшие гиперпараметры с помощью grid search.

In [10]:
forest_params = {'n_estimators': np.arange(1, 30, 5), 
                 'min_samples_leaf': np.arange(1, 20, 5),
                 'criterion': ['gini','entropy'],
                 'max_depth': np.arange(10, 40, 10)}

forest_grid = GridSearch(RandomForest, forest_params, scoring='accuracy', cv=3)
forest_grid.fit(X, y)
print(forest_grid.best_score_)
print(forest_grid.best_params_)

0.9539999290870899
[['n_estimators', 1], ['min_samples_leaf', 1], ['criterion', 'gini'], ['max_depth', 10]]


Таким образом, при следующих гиперпараметрах [['n_estimators', 1], ['min_samples_leaf', 1], ['criterion', 'gini'], ['max_depth', 10]] модель дает наибольшую точность

Метод оценивания информативности признаков заключается в следующем: ошибку Qn базового дерева bn оценивают по out-of-bag выборке и запоминают. После этого признак j превращают в абсолютно бесполезный, шумовой: в матрицу «объекты-признаки» все значения в столбце j перемешивают. Затем то же самое дерево bn применяют к данной выборке с перемешанным признаком j и оценивают качество дерева на out-of-bag-подвыборке. Q′n — ошибка out-of-bag-подвыборке, она будет тем больше, чем сильнее дерево использует признак j. Если он активно используется в дереве, то ошибка сильно уменьшится, поскольку значение данного признака испорчено. Если же данный признак совершенно не важен для дерева и не используется в нем, то ошибка практически не изменится. Таким образом, информативность признака j оценивают как разность
Q ′n − Q n .
Далее эти информативности усредняют по всем деревьям случайного леса, и чем больше будет среднее значе- ние, тем важнее признак. На практике оказывается, что информативности, вычисленные описанным образом, и информативности, вычисленные как сумма уменьшения критерия информативности, оказываются очень связаны между собой.


In [ ]:
print(forest.get_feature_informativeness(X, y))

[ 4  0  1  5  7  2  3  9  8 11 10  6 12]


Мы получили вектор индексов признаков в порядке убывания их значимости. Таким образом, Balance, Age, NumOfProducts, EstimatedSalary, IsActiveMember являются самыми информативными (5 первых признаков из вектора). 